In [1]:
import moon.data
import moon.problem

In [2]:
probs = moon.data.read_problems('data/cleaned_probs.csv')

In [14]:
import numpy as np

x = np.array([p.array for p in probs])
y = np.array([p.grade.ordinal for p in probs])

print(x.shape, y.shape)

(30991, 18, 11) (30991, 17)


In [22]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K

class OrdinalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='ordinal_acc_within_k', k=0, threshold=0.5, **kwargs):
        super(OrdinalAccuracy, self).__init__(name=name, **kwargs)
        self.k = k
        self.threshold = threshold
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # See section 2.3: http://calla.rnet.missouri.edu/cheng_courses/rank.pdf
        pred_thresh = tf.cast(y_pred >= self.threshold, 'int8')
        true_thresh = tf.cast(y_true >= self.threshold, 'int8')
        pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
        true_idx = tf.argmin(true_thresh, axis=-1) - 1
        distance_bools = (tf.abs(pred_idx - true_idx) <= self.k)
        correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
        incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
        self.total.assign_add(correct)
        self.count.assign_add(correct + incorrect)
    
    def result(self):
        return self.total / self.count

    
# α = 0.3
p = .4
input_shape = moon.problem.Problem.GRID_SHAPE
hiddens = [16, 16]
hidden_activation = 'swish'
output_shape = moon.problem.Grade.N_GRADES
output_activation = 'sigmoid'
loss = 'mse'
adam_lr = 1e-4

metrics = [OrdinalAccuracy(name='acc0', k=0), OrdinalAccuracy(name='acc1', k=1), OrdinalAccuracy(name='acc2', k=2)]
optim = tf.keras.optimizers.Adam(lr=adam_lr)

In [23]:
in_x = layers.Input(shape=input_shape)
features = layers.Flatten()(in_x)
for nodes in hiddens:
    features = layers.Dense(nodes, activation=hidden_activation)(features)
    # features = layers.ReLU(α)(features)
    if p > 0: features = layers.Dropout(p)(features)
out_y = layers.Dense(output_shape, activation=output_activation)(features)

model = tf.keras.Model(in_x, out_y, name='Feedforward_classifier')
model.compile(loss=loss, optimizer=optim, metrics=metrics)
# model.summary()

In [17]:
split = 0.2
indices = np.arange(len(probs))
np.random.shuffle(indices)
n_test = int(len(probs) * split)

x_train, y_train = x[indices[n_test:]], y[indices[n_test:]]
x_test, y_test = x[indices[0:n_test]], y[indices[0:n_test]]
print(x_train.shape, x_test.shape)

(24793, 18, 11) (6198, 18, 11)


In [24]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=4, verbose=1)]
batch_size = 64
max_epochs = 300

history = model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=max_epochs, callbacks=callbacks)

# [256, 128, 64], p=.4, flattened, 28e
# loss: 0.0396 - acc0: 0.4144 - acc1: 0.7597 - acc2: 0.9229 - val_loss: 0.0414 - val_acc0: 0.3925 - val_acc1: 0.7375 - val_acc2: 0.9140

# [16, 16], p=.4, flattened, 76e
# loss: 0.0446 - acc0: 0.3766 - acc1: 0.7139 - acc2: 0.8944 - val_loss: 0.0416 - val_acc0: 0.3879 - val_acc1: 0.7438 - val_acc2: 0.9138

# [16, 16], p=.4, flattened, lr 1e-4, 202e
# loss: 0.0463 - acc0: 0.3726 - acc1: 0.6968 - acc2: 0.8863 - val_loss: 0.0429 - val_acc0: 0.3987 - val_acc1: 0.7257 - val_acc2: 0.9037

# [32], p=.4, flattened, 31e
# loss: 0.0431 - acc0: 0.3896 - acc1: 0.7303 - acc2: 0.9021 - val_loss: 0.0416 - val_acc0: 0.4055 - val_acc1: 0.7385 - val_acc2: 0.9096

# [256, 128, 64], p=.4, 3d, 24e
# loss: 0.0409 - acc0: 0.4075 - acc1: 0.7475 - acc2: 0.9123 - val_loss: 0.0421 - val_acc0: 0.3945 - val_acc1: 0.7389 - val_acc2: 0.9076

# [64, 64], p=.4, 3d, 32e
# loss: 0.0420 - acc0: 0.4018 - acc1: 0.7372 - acc2: 0.9087 - val_loss: 0.0419 - val_acc0: 0.3992 - val_acc1: 0.7396 - val_acc2: 0.9080

# [64], p=.4, 3d, 44e
# loss: 0.0405 - acc0: 0.4103 - acc1: 0.7505 - acc2: 0.9152 - val_loss: 0.0416 - val_acc0: 0.4016 - val_acc1: 0.7402 - val_acc2: 0.9092

# [32], p=.4, 3d, 59e
# loss: 0.0418 - acc0: 0.4037 - acc1: 0.7396 - acc2: 0.9084 - val_loss: 0.0420 - val_acc0: 0.4045 - val_acc1: 0.7314 - val_acc2: 0.9093

Epoch 1/300
388/388 [==============================] - 1s 3ms/step - loss: 0.2387 - acc0: 0.0385 - acc1: 0.1199 - acc2: 0.2048 - val_loss: 0.2173 - val_acc0: 0.1912 - val_acc1: 0.4450 - val_acc2: 0.7757
Epoch 2/300
388/388 [==============================] - 1s 2ms/step - loss: 0.1775 - acc0: 0.1289 - acc1: 0.3719 - acc2: 0.5724 - val_loss: 0.1204 - val_acc0: 0.1481 - val_acc1: 0.4155 - val_acc2: 0.8414
Epoch 3/300
388/388 [==============================] - 1s 2ms/step - loss: 0.1141 - acc0: 0.1671 - acc1: 0.4571 - acc2: 0.7098 - val_loss: 0.0802 - val_acc0: 0.1481 - val_acc1: 0.4397 - val_acc2: 0.8408
Epoch 4/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0933 - acc0: 0.1786 - acc1: 0.4788 - acc2: 0.7352 - val_loss: 0.0722 - val_acc0: 0.1488 - val_acc1: 0.4945 - val_acc2: 0.8401
Epoch 5/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0850 - acc0: 0.1899 - acc1: 0.4898 - acc2: 0.7393 - val_loss: 0.0695 - val_acc0: 0.1484 - val_acc1: 0.5342 - va

388/388 [==============================] - 1s 2ms/step - loss: 0.0516 - acc0: 0.3381 - acc1: 0.6579 - acc2: 0.8552 - val_loss: 0.0472 - val_acc0: 0.3777 - val_acc1: 0.6939 - val_acc2: 0.8766
Epoch 42/300
388/388 [==============================] - 1s 3ms/step - loss: 0.0516 - acc0: 0.3368 - acc1: 0.6540 - acc2: 0.8555 - val_loss: 0.0472 - val_acc0: 0.3782 - val_acc1: 0.6944 - val_acc2: 0.8767
Epoch 43/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0512 - acc0: 0.3407 - acc1: 0.6593 - acc2: 0.8580 - val_loss: 0.0471 - val_acc0: 0.3796 - val_acc1: 0.6951 - val_acc2: 0.8779
Epoch 44/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0513 - acc0: 0.3408 - acc1: 0.6565 - acc2: 0.8566 - val_loss: 0.0470 - val_acc0: 0.3785 - val_acc1: 0.6933 - val_acc2: 0.8782
Epoch 45/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0510 - acc0: 0.3409 - acc1: 0.6597 - acc2: 0.8577 - val_loss: 0.0470 - val_acc0: 0.3816 - val_acc1: 0.6936 - val_acc2: 

388/388 [==============================] - 1s 2ms/step - loss: 0.0490 - acc0: 0.3565 - acc1: 0.6757 - acc2: 0.8686 - val_loss: 0.0453 - val_acc0: 0.3888 - val_acc1: 0.7081 - val_acc2: 0.8909
Epoch 82/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0488 - acc0: 0.3562 - acc1: 0.6770 - acc2: 0.8696 - val_loss: 0.0452 - val_acc0: 0.3882 - val_acc1: 0.7093 - val_acc2: 0.8913
Epoch 83/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0486 - acc0: 0.3572 - acc1: 0.6793 - acc2: 0.8715 - val_loss: 0.0452 - val_acc0: 0.3892 - val_acc1: 0.7089 - val_acc2: 0.8921
Epoch 84/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0487 - acc0: 0.3534 - acc1: 0.6810 - acc2: 0.8716 - val_loss: 0.0451 - val_acc0: 0.3896 - val_acc1: 0.7097 - val_acc2: 0.8919
Epoch 85/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0487 - acc0: 0.3537 - acc1: 0.6794 - acc2: 0.8717 - val_loss: 0.0451 - val_acc0: 0.3895 - val_acc1: 0.7096 - val_acc2: 

388/388 [==============================] - 1s 3ms/step - loss: 0.0475 - acc0: 0.3619 - acc1: 0.6913 - acc2: 0.8798 - val_loss: 0.0438 - val_acc0: 0.3963 - val_acc1: 0.7183 - val_acc2: 0.9001
Epoch 122/300
388/388 [==============================] - 1s 3ms/step - loss: 0.0472 - acc0: 0.3653 - acc1: 0.6916 - acc2: 0.8799 - val_loss: 0.0438 - val_acc0: 0.3969 - val_acc1: 0.7194 - val_acc2: 0.9001
Epoch 123/300
388/388 [==============================] - 1s 3ms/step - loss: 0.0474 - acc0: 0.3653 - acc1: 0.6900 - acc2: 0.8806 - val_loss: 0.0437 - val_acc0: 0.3971 - val_acc1: 0.7188 - val_acc2: 0.9003
Epoch 124/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0475 - acc0: 0.3637 - acc1: 0.6923 - acc2: 0.8779 - val_loss: 0.0437 - val_acc0: 0.3967 - val_acc1: 0.7189 - val_acc2: 0.9005
Epoch 125/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0471 - acc0: 0.3678 - acc1: 0.6943 - acc2: 0.8815 - val_loss: 0.0436 - val_acc0: 0.3975 - val_acc1: 0.7202 - val_ac

388/388 [==============================] - 1s 2ms/step - loss: 0.0467 - acc0: 0.3665 - acc1: 0.6956 - acc2: 0.8819 - val_loss: 0.0431 - val_acc0: 0.3972 - val_acc1: 0.7241 - val_acc2: 0.9035
Epoch 162/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0467 - acc0: 0.3667 - acc1: 0.6987 - acc2: 0.8831 - val_loss: 0.0432 - val_acc0: 0.3985 - val_acc1: 0.7247 - val_acc2: 0.9038
Epoch 163/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0467 - acc0: 0.3625 - acc1: 0.6976 - acc2: 0.8826 - val_loss: 0.0432 - val_acc0: 0.4025 - val_acc1: 0.7244 - val_acc2: 0.9027
Epoch 164/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0466 - acc0: 0.3672 - acc1: 0.6966 - acc2: 0.8836 - val_loss: 0.0431 - val_acc0: 0.3975 - val_acc1: 0.7241 - val_acc2: 0.9035
Epoch 165/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0465 - acc0: 0.3644 - acc1: 0.6948 - acc2: 0.8844 - val_loss: 0.0431 - val_acc0: 0.3988 - val_acc1: 0.7259 - val_ac

388/388 [==============================] - 1s 2ms/step - loss: 0.0466 - acc0: 0.3623 - acc1: 0.6981 - acc2: 0.8840 - val_loss: 0.0430 - val_acc0: 0.3990 - val_acc1: 0.7247 - val_acc2: 0.9032
Epoch 202/300
388/388 [==============================] - 1s 2ms/step - loss: 0.0463 - acc0: 0.3726 - acc1: 0.6968 - acc2: 0.8863 - val_loss: 0.0429 - val_acc0: 0.3987 - val_acc1: 0.7257 - val_acc2: 0.9037
Epoch 00202: early stopping


In [78]:
# print(np.round(model.predict(x_test[99][np.newaxis, :]) * 100))
# print(y_test[99])
y_true = y_test[[2, 4, 8, 16, 32]]
y_pred = model.predict(x_test[[2, 4, 8, 16, 32]])

In [79]:
pred_thresh = tf.cast(y_pred >= .5, 'int8')
true_thresh = tf.cast(y_true >= .5, 'int8')
print(pred_thresh, true_thresh)

# Since tf.argmin returns the lowest index possible, this gets us the index of the last True value.
# Note that this is -1 if the network doesn't many any prediction at all.
pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
true_idx = tf.argmin(true_thresh, axis=-1) - 1
print(pred_idx, true_idx)

distance_bools = tf.abs(pred_idx - true_idx) <= 0
correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
print(distance_bools, correct, incorrect)

print(correct / (correct+incorrect))

tf.Tensor(
[[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8) tf.Tensor(
[[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8)
tf.Tensor([7 8 7 8 6], shape=(5,), dtype=int64) tf.Tensor([8 9 6 7 6], shape=(5,), dtype=int64)
tf.Tensor([False False False False  True], shape=(5,), dtype=bool) tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(0.2, shape=(), dtype=float32)
